In [3]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
from gmm_dataset import *
from util_data import *
from util_hmm_variational_gibbs import *
from util_plots import *
from scipy.stats import invwishart, dirichlet
from torch.distributions.dirichlet import Dirichlet
sys.path.append('/home/hao/Research/probtorch/')
from probtorch.util import expand_inputs
import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.5.0a0+3bb8c5e cuda: True


In [22]:
T = 100
K = 4
D = 2
num_particles_rws = 50
mcmc_steps = 2
num_particles_smc = 30
NUM_HIDDEN = 128
NUM_LATENTS = K*K
NUM_OBS = 2 * K
# training parameters
NUM_SAMPLES = 1
# BATCH_SIZE = 100
NUM_EPOCHS = 80
LEARNING_RATE = 1e-3
CUDA = False

num_series = 1
dt = 1 ## \delta t
Boundary = 50 ## unit box is amplified by this value, centered at origin
signal_noise_ratio = 0.15 # the noise is multiplied by this param
training_epochs = 50 
D = 2
## Model parameters
K =  4## number of hidden states

In [23]:
## generate time series
x0, y0, init_v, init_v_rand_dir = intialization(T, num_series, Boundary)
mu_ks = np.zeros((num_series, K, D))
final_covs = np.zeros((num_series, K, D, D))
STATEs = np.zeros((num_series, T+1, 4))
Disps = np.zeros((num_series, T, D))
As_true = np.zeros((num_series, 4, 4))

for s in range(num_series):
    init_state = np.array([x0[s], y0[s], init_v_rand_dir[s, 0], init_v_rand_dir[s, 1]])
    STATEs[s], Disps[s], As_true[s] = generate_data(T, dt, init_state, Boundary, signal_noise_ratio)

Y = Disps[0]

In [30]:
# plt.plot(Y[:,0], Y[:,1], 'ro')
init_v_rand_dir

array([[-7.82429537, -6.22739126]])

In [ ]:
## return samples in order to compute the weights and 
class Encoder(nn.Module):
    def __init__(self, num_obs=NUM_OBS,
                       num_hidden=NUM_HIDDEN,
                       num_latents=NUM_LATENTS):
        super(self.__class__, self).__init__()
        self.enc_hidden = nn.Sequential(
            nn.Linear(num_obs, num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, num_hidden),
            nn.ReLU())
        self.latent_dir = nn.Sequential(
            nn.Linear(num_hidden, num_latents))
        
    def forward(self, obs, num_particles):
        A_samples = torch.zeros((K, K))
        hidden = self.enc_hidden(obs)
        latents_dirs = torch.exp(self.latent_dir(hidden)).sum(0).view(K, K)
        latents_dirs_norm = latents_dirs / latents_dirs.sum() * (T-1)
        for k in range(K):
            A_samples[k] = Dirichlet(latents_dirs_norm[k]).sample()
        return latents_dirs_norm, A_samples

In [ ]:
def initialize():
    enc = Encoder()
    if CUDA:
        enc.cuda()
    optimizer =  torch.optim.Adam(list(enc.parameters()),lr=LEARNING_RATE)    
    return enc, optimizer

alpha_init_0, alpha_trans_0, m_0, beta_0, nu_0, W_0 = pirors(Y, T, D, K)
enc, optimizer = initialize()
## initialization
# cov_ks = torch.zeros((K, D, D))
# mu_ks = torch.zeros((K, D))
# for k in range(K):
# ## sample mu_k and Sigma_k randomly
#     cov_ks[k] = torch.from_numpy(invwishart.rvs(df=nu_0, scale=W_0.data.numpy())).float()
#     mu_ks[k] = MultivariateNormal(loc=m_0, covariance_matrix=cov_ks[k] / beta_0).sample()
# Pi = Dirichlet(alpha_init_0).sample()
# A_samples = initial_trans(alpha_trans_0, K)

In [ ]:
KLs = []
KL_ests = []
log_p_conds = []
log_qs = []
ESSs = []

Pi = Pi_true
mu_ks = mu_true
cov_ks = cov_true
# Z_ret = Zs_true
# Z_ret_pairwise = torch.cat((Z_ret[:T-1].unsqueeze(0), Z_ret[1:].unsqueeze(0)), 0).transpose(0, 1).contiguous().view(T-1, 2*K)

for epoch in range(NUM_EPOCHS):
    time_start = time.time()
    optimizer.zero_grad()
    log_weights_rws = torch.zeros(num_particles_rws)
    log_qs = torch.zeros(num_particles_rws)
    kl = 0.0
    for l in range(num_particles_rws):
#         A_samples = initial_trans(alpha_trans_0, K)
        A_samples = A_true
        Zs, log_weights, log_normalizer = smc_hmm(Pi, A_samples, mu_ks, cov_ks, Y, T, D, K, num_particles_smc)
        Z_ret = resampling_smc(Zs, log_weights)
        log_weight_rws = log_normalizer
        Z_ret_pairwise = torch.cat((Z_ret[:T-1].unsqueeze(0), Z_ret[1:].unsqueeze(0)), 0).transpose(0, 1).contiguous().view(T-1, 2*K)
        for m in range(mcmc_steps):
            A_prev = A_samples
            latents_dirs, A_samples = enc(Z_ret_pairwise, 1)
            log_p_joint_curr = log_joint(alpha_init_0, alpha_trans_0, nu_0, W_0, m_0, beta_0, Z_ret, Pi, A_samples, mu_ks, cov_ks, Y, T, D, K).detach().item()
#             log_p_joint_prev = log_joint(alpha_init_0, alpha_trans_0, nu_0, W_0, m_0, beta_0, Z_ret, Pi, A_prev, mu_ks, cov_ks, Y, T, D, K).detach().item() 
            log_p_smc = log_joint_smc(Z_ret, Pi, A_samples, mu_ks, cov_ks, Y, T, D, K)
            log_q_curr = log_q_hmm(latents_dirs, A_samples, K, num_particles=1).detach()
            log_q_prev = log_q_hmm(latents_dirs, A_prev, K,num_particles=1).detach()
            
            Zs, log_weights, log_normalizer = csmc_hmm(Z_ret, Pi, A_samples, mu_ks, cov_ks, Y, T, D, K, num_particles_smc)
            Z_ret = resampling_smc(Zs, log_weights)
            Z_ret_pairwise = torch.cat((Z_ret[:T-1].unsqueeze(0), Z_ret[1:].unsqueeze(0)), 0).transpose(0, 1).contiguous().view(T-1, 2*K)

            log_weight_rws += log_p_joint_curr - log_p_smc - log_q_curr + log_q_prev + log_normalizer
              
        log_weights_rws[l] = log_weight_rws.detach()     

  
        alpha_trans_hat = alpha_trans_0 + pairwise(Z_ret, T).sum(0)
        kl += kl_dirichlets(alpha_trans_0, latents_dirs, Z_ret, T, K)      
        log_qs[l] = log_q_hmm(latents_dirs, A_samples, K, num_particles=1)   
    kl /= num_particles_rws
        
    log_weights_rws = log_weights_rws - log_sum_exp(log_weights_rws)
    weights_rws = torch.exp(log_weights_rws)
    ess = (1. / (weights_rws ** 2 ).sum()).item()
    loss_infer = - torch.mul(weights_rws, log_qs).sum()
#     kl_est = torch.mul(weights_rws, log_p_conds - log_qs).sum().detach().item()
    loss_infer.backward()
    KLs.append(kl.item())
#     KL_ests.append(kl_est)
    ESSs.append(ess)
    optimizer.step()
#     A_samples = A_samples.detach()
    time_end = time.time()
    print('epoch : %d, KL : %f, loss_infer : %f (%ds)' % (epoch, kl, loss_infer, time_end - time_start))

In [ ]:
# plot_kl_est(np.array(KLs))

In [ ]:
alpha_trans_0 + pairwise(Zs_true, T).sum(0)

In [ ]:
learned_dicichlet_post = latents_dirs
true_dirichlet_post = alpha_trans_0 + pairwise(Zs_true, T).sum(0)
print('variational : ')
print(learned_dicichlet_post)
print('conjugate posterior :')
print(true_dirichlet_post)

In [ ]:
plt.plot(np.array(ESSs) / num_particles_rws)